This notebook shows how to treat surface reactions in reacting systems where diffusion is a much faster process in comparison to the chemical reactions (surface is treated as a "homogenized" concentration).

In [ ]:
from collections import defaultdict
from chempy import ReactionSystem, Species, Reaction
from chempy.units import SI_base_registry, default_units as u
from chempy.kinetics.ode import (
    _create_odesys as create_odesys,
)

In [ ]:
ads_token = '(ads)'
phases = {'(aq)': 0, ads_token: 1}
ads_comp = -1
vacancy = Species('vacancy(ads)', composition={ads_comp: 1},
                  phase_idx=phases[ads_token])

def substance_factory(name):
    if name == vacancy.name:
        return vacancy
    s = Species.from_formula(name, phases=phases)
    if name.endswith(ads_token):
        s.composition[ads_comp] = 1
    return s

rsys = ReactionSystem.from_string("""
vacancy(ads) + H2O2 -> H2O2(ads); 'k_ads_H2O2'
H2O2(ads) + vacancy(ads) -> 2 OH(ads); 'k_split_H2O2ads'
OH(ads) + H2O2 -> H2O + HO2 + vacancy(ads); 'k_abst_OHads_H2O2'
HO2 + HO2 -> O2 + H2O2; 'k_disprop_HO2'
""", substance_factory=substance_factory)
rsys

In [ ]:
odesys, extra = create_odesys(rsys, unit_registry=SI_base_registry)

In [ ]:
result, _ = extra['unit_aware_solve'](
    42*u.s, defaultdict(lambda: 0*u.M, H2O2=1*u.molar),
    dict(
        k_ads_H2O2=.1/u.M/u.s,
        k_split_H2O2ads=.2/u.M/u.s,
        k_abst_OHads_H2O2=.3/u.M/u.s,
        k_disprop_HO2=.4/u.M/u.s
    )
)